In [ ]:
import numpy as np
from loadMNIST import LoadMNIST
import matplotlib.pyplot as plt
%matplotlib inline

## A DIY classifier
In this notebook, we'll build a simple classifer to discriminate between 1's and 0's in the MNIST database.

First, we'll construct a simple 2D feature space by projecting digit images onto hand-selected 0 and 1 templates. 

We'll then design a linear classifier by hand, just by looking at the distribution of MNIST images in our simple feature space.

### The data
We'll use the MNIST dataset for this exercise. First we'll load up the images and normalize them.

In [ ]:
##load all MNIST images, then select out the 0 and 1 images

mnist_trn_img = '/mnt/fast/MNIST/train-images-idx3-ubyte'
mnist_trn_lab = '/mnt/fast/MNIST/train-labels-idx1-ubyte'
mnist_val_img = '/mnt/fast/MNIST/t10k-images-idx3-ubyte'
mnist_val_lab = '/mnt/fast/MNIST/t10k-labels-idx1-ubyte'

mnist_trn_data = LoadMNIST(mnist_trn_img)
mnist_val_data = LoadMNIST(mnist_val_img)
mnist_trn_label = LoadMNIST(mnist_trn_lab)
mnist_val_label = LoadMNIST(mnist_val_lab)

trn_ones = mnist_trn_data[mnist_trn_label==1].astype('float32')
val_ones = mnist_val_data[mnist_val_label==1].astype('float32')

trn_zeros = mnist_trn_data[mnist_trn_label==0].astype('float32')
val_zeros = mnist_val_data[mnist_val_label==0].astype('float32')

In [ ]:
##sanity check
plt.subplot(1,4,1)
plt.imshow(trn_ones[0],cmap='gray')
plt.title('one')
plt.subplot(1,4,2)
plt.imshow(trn_zeros[0],cmap='gray')
plt.title('zero')
plt.subplot(1,4,3)
plt.imshow(val_ones[0],cmap='gray')
plt.title('one')
plt.subplot(1,4,4)
plt.imshow(val_zeros[0],cmap='gray')
plt.title('zero')

pixel_res = trn_ones.shape[1]
n_trn_ones = trn_ones.shape[0]
n_val_ones = val_ones.shape[0]
n_trn_zeros = trn_zeros.shape[0]
n_val_zeros = val_zeros.shape[0]

print 'there are %d ones and %d zeros in the training set' %(n_trn_ones,n_trn_zeros)
print 'there are %d ones and %d zeros in the validation set' %(n_val_ones,n_val_zeros)


It is generally a good idea to normalize the input data, so that feature space is not dominated by the "brightness" of images, but rather by their shape. To do this, we first subtract the mean from each image, then divide each image by it's magnitude

In [ ]:
##normalize images
##note the use of np.newaxis to take advantage of broadcasting rules!
for arr in [trn_ones, trn_zeros, val_ones, val_zeros]:
    arr -= np.mean(arr,axis=(1,2))[:,np.newaxis,np.newaxis]  
    arr /= np.sqrt(np.sum(arr**2,axis=(1,2)))[:,np.newaxis,np.newaxis]

### The feature space
We'll construct a simple linear transform that maps each MNIST image into a point in a 2D feature space.

The axes of this feature space will correspond to a 0-template and a 1-template.

Select two images from the training data to be "templates"

In [ ]:
one_temp_idx = np.random.randint(0,high=n_trn_ones)    ##random template selection
zero_temp_idx = np.random.randint(0,high=n_trn_zeros)
one_temp = trn_ones[one_temp_idx]
zero_temp = trn_zeros[zero_temp_idx]

##view your templates
plt.subplot(1,2,1)
plt.imshow(one_temp,cmap='gray')
plt.title('template for one')
plt.subplot(1,2,2)
plt.imshow(zero_temp,cmap='gray')
plt.title('template for zero')

Let $\mathbf{t_0}$ be the zero template, and $\mathbf{t_1}$ be the one template. Let $\mathbf{s}$ be an image in the training set. Map all training data images (both zeros and ones) into the 2D feature space defined

$\phi(\mathbf{s}) = (\mathbf{s} \cdot \mathbf{t_0}, \mathbf{s} \cdot \mathbf{t_1})$

In [ ]:
##first, we reshape the templates and stack them into a matrix
temps = np.array([zero_temp.ravel() , one_temp.ravel()]).T
print temps.shape

##next we project each of the temp
phi_s_ones = trn_ones.reshape((n_trn_ones,pixel_res**2)).dot(temps)
print phi_s_ones.shape

phi_s_zeros = trn_zeros.reshape((n_trn_zeros,pixel_res**2)).dot(temps)
print phi_s_zeros.shape


In [ ]:
##view the projections, assigning different colors to each class
plt.scatter(phi_s_ones[:,0], phi_s_ones[:,1],c=[1,0,0],label='ones', alpha = 1)
plt.scatter(phi_s_zeros[:,0], phi_s_zeros[:,1],c=[0,0,1],label='zeros', alpha = 0.5)
plt.legend()
plt.gca().set_aspect('equal', adjustable='box')
plt.xlabel('projection onto zero template')
plt.ylabel('projection onto one template')

Not surprisingly, images of ones hug the "ones" axis, and images of zeros hug the "zeros" axis. Now let's try to hand-code a discriminant surface into this feature space that separates out the two classes of images.

### Hand-coding a classifier

In class we learned that linear discriminant functions:

$y(\mathbf{x}) = \mathbf{w} \cdot \mathbf{x} + b$

implicitly define a discriminant surface as all point $\left \{ \mathbf{x}: y(\mathbf{x})=0 \right \}$, that is, all points where the discriminant function is zero.

For convenience, let's write a simple function called "view_classes" that takes a weight vector (1,2) and a bias (scalar) and colors the 2D plain according to class asignment.

In [ ]:
    X,Y = np.meshgrid(np.linspace(0,1,num=10),
                      np.linspace(0,1,num=10))


In [ ]:
np.array([X.ravel(),Y.ravel()]).shape

In [ ]:
def view_class(weight_vector, bias, view_bounds=(-.5,1.5), view_res=100):
    ##construct a viewing plane
    X,Y = np.meshgrid(np.linspace(view_bounds[0],view_bounds[1],num=view_res),
                      np.linspace(view_bounds[0],view_bounds[1],num=view_res))
    
    points = np.array([X.ravel(),Y.ravel()]) ##2 x view_res**2
    
    ##evaluate discriminate function
    disc = weight_vector.dot(points)+bias
    
    ##threshold
    disc = (disc >= 0)
    
    ##reshape disc. function values into a matrix and view
    plt.pcolor(X,Y,disc.reshape((view_res,view_res)),cmap='jet')
    plt.gca().set_aspect('equal', adjustable='box')
    
    
    return disc

Now let's try out a few values of w and b to see if they'll give us decent classification

In [ ]:
##test with w = (-1,1), b = 1
_ = view_class(np.array([-1,.5]), -.28)

x_color = np.array([.8,.3,.6])  ##pink vector
y_color = np.array([.2,.6,.8])  ##blue vector

plt.scatter(phi_s_ones[:,0], phi_s_ones[:,1],c=x_color,label='ones', alpha = 1)
plt.scatter(phi_s_zeros[:,0], phi_s_zeros[:,1],c=y_color,label='zeros', alpha = 0.5)
plt.legend()
plt.gca().set_aspect('equal', adjustable='box')
plt.xlabel('projection onto zero template')
plt.ylabel('projection onto one template')